# 방법 1
### deque를 사용한다
> * 주요 메소드
* rotate(-1) 왼쪽으로 한칸씩 값을 이동. 즉 0, 1, 2, 3, 4, 5 일때 1, 2, 3, 4, 5, 0이 된다.
* 만약 3번째 놈이 죽어야 하면 -1을 세번하고 맨 앞에 사람을 pop하면 원하는 놈들을 뺄 수 있다.

### 문제점
가장 큰 문제점은 맨 앞에 rotate 할 경우 deque가 linked list가 아닌 배열이기 때문에 재배치하는데 n만큼의 시간이 걸린다! 그래서 시간초과가 난다. 또 여기서 주목할 점은 popleft는 거의 O(1)이라고 한다. 그 이유는 

In [4]:
import time
from collections import deque
a = time.time() 
t = 0.0
for _ in range(int(input())):
    n, k = map(int, input().split())
    a = time.time()
    number = deque()
    result = 0
    for i in range(n):
        number.append(str(i+1))
    # 숫자 다 넣어짐
    i = 0
    number.popleft()
    while len(number) > 2:
        i +=1
        if i != k:
            number.rotate(-1)
        else:
            i = 0
            number.popleft()
    number = list(sorted(number))
    result = ' '.join(number)
    print(result)
    b = time.time()
    t += (b-a)

print(t)

2
6 3
3 5
40 3
11 26
0.0


주피터 노트북에서는 0.0으로 나오지만 vscode를 사용한 결과 수행시간이 0.001991748809814453가 나온다. 그리고 위 식은 알고 스팟에서 시간 초과가 나온다(백준에 조세푸스 문제도 0번 1번은 풀리지만 2번부턴 시간 초과가 나온다)

# 방법2
### Linked list를 만들어준다.
> * 직접 linked list를 만들어서 시간을 봐보겠다. 이론상 시간초과가 나오면 안되는데 시간 초과가 난다....

아래는 class 부분 구현한거다 겁나 불편해서 어차피 못 읽을거라고 생각한다.


In [ ]:
class node:
    def __init__(self, data,prev = None, nex= None):
        self.data = data
        self.next = nex
        self.previous = prev

class linked:
    def __init__(self):
        self.head = None
        self.current = None
        self.len = 0
        self.iter = None
    
    def move_right(self):
        self.iter = self.iter.next
    
    def move_left(self):
        self.iter = self.iter.previous

    def append(self, value):
        if self.len == 0:
            self.current = self.head = node(data = value)
            self.current.next = self.current.previous = self.current
            self.iter = self.head
        else:
            self.current.next = node(data = value, prev = self.current, nex = self.head)
            self.head.previous = self.current.next
            self.current = self.current.next
        self.len +=1    
    
    def getlen(self):
        return self.len

    def delete(self):
        assert (self.len != 0), "no node left"
        self.len -=1
        self.iter.previous.next = self.iter.next
        self.iter.next.previous = self.iter.previous
        a = self.iter
        self.iter = self.iter.next
        if a == self.head:
            self.head = self.head.next
        value = a.data
        del a
        return value

그 외의 코드 부분이다. 책에서 나온 개념을 그대로 사용하였는데도 시간초과가 뜬다.

In [ ]:
import time

t = 0.0
           
for _ in range(int(input())):
    link = linked()
    result = []
    n, k = map(int, input().split())
    a = time.time()
    for i in range(1,n):
        link.append(str(i+1))
    cnt = 0
    while link.getlen() > 2:
        cnt+=1
        if cnt == k:
            cnt = 0
            link.delete()
        else:
            link.move_right()
    while link.getlen() != 0:
        result.append(link.delete())
    result.sort()
    result = ' '.join(result)
    print(result)

    del link
    b = time.time()
    t += (b-a)
print(t)

실제 같은 데이터를 넣고 vscode에서 돌려본 결과 0.0028448104858398438로 기존보다도 크게 나온다. 전체 크기가 많이 변하면 달라질 수도 있겠지만 어쨌든 알고스팟에서 시간초과가 떴다는 것은 이것도 안된다는 건데 왜 이럴깡?

# 방법3
### <strike> deque에서 반대로 접근하여 제거할때 O(1)만큼 사용하기?!</strike>
> * <strike> 이거 정리하다가 생각난 방법인데 deque의 문제가 맨 앞에 값을 지우기 때문이라면 배열 맨 뒤부터 접근하여 rotate(1)을 하면서 맨 뒷 값을 지우는건 어떨까? 사실 rotate 자체가 O(n)이라서 큰 효과가 없을 것 같지만 한번 해보지 뭐 안 어려우니깐.</strike>

popleft가 O(n)인 줄 알고 했는데 아니라고 합니다. 그래서 같은 값이 나올 줄 알았는데 그건 아니다?!

In [ ]:
import time
from collections import deque
t = 0.0
for _ in range(int(input())):
    n, k = map(int, input().split())
    a = time.time()
    number = deque()
    result = 0
    for i in range(n,1,-1):
        number.append(str(i))
    # 숫자 다 넣어짐

    i = 0
    while len(number) > 2:
        i +=1
        if i != k:
            number.rotate(1)
        else:
            i = 0
            number.pop()
    number = list(sorted(number))
    result = ' '.join(number)
    print(result)
    b = time.time()
    t+= (b-a)
print(t)

이건 결과 0.0041484832763671875이다. 더 늘었다?! ㄴㅇㄱ

컴터 상태에 따라 다른듯 이게 0.0019나올 때도 있고 위에 방법 1이 0.0028 이렇게 나올때도 있넹.. ㅡㅡ